# Working with bits

* To utilize certain features of microcontrollers, often times we need to turn on or off certain bits of certain memory location.

* The pointer of C/C++ is a suitable tool for this type of task.

* However, the smallest unit of memory that we can change is a byte; hence we may need to first read the current status of the particular byte, change that one bit, and then write the *bit pattern* to that byte.

## Revisiting `struct` and `union`

* `struct` is a group of data.  An array is also a group of data.  Difference is, to access data in `struct`, we would use *field name*s; for arrays, indices.

* Also, *field*s of struct may have different types; for arrays, all same types.

* For each field of struct, we may specify its length in **bits**.

* Followinging example shows a 16bit color information.

``` C++
struct high_color_tag{
    unsigned char red:5;
    unsigned char green:6;
    unsigned char blue:5;
};
```

* We can see that the sum of bits is `5 + 6 + 5 = 16`; two bytes.